Automatic differentiation with torch.autograd

In [1]:
%matplotlib inline
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001E043553A30>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001E0435538E0>


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1059, 0.1769, 0.2907],
        [0.1059, 0.1769, 0.2907],
        [0.1059, 0.1769, 0.2907],
        [0.1059, 0.1769, 0.2907],
        [0.1059, 0.1769, 0.2907]])
tensor([0.1059, 0.1769, 0.2907])


In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [6]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [8]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300314  [    0/60000]
loss: 2.296514  [ 6400/60000]
loss: 2.284238  [12800/60000]
loss: 2.278248  [19200/60000]
loss: 2.265664  [25600/60000]
loss: 2.245462  [32000/60000]
loss: 2.248735  [38400/60000]
loss: 2.235715  [44800/60000]
loss: 2.240423  [51200/60000]
loss: 2.195017  [57600/60000]
Test Error: 
 Accuracy: 37.7%, Avg loss: 0.034677 

Epoch 2
-------------------------------
loss: 2.219741  [    0/60000]
loss: 2.208793  [ 6400/60000]
loss: 2.189236  [12800/60000]
loss: 2.196109  [19200/60000]
loss: 2.141999  [25600/60000]
loss: 2.115577  [32000/60000]
loss: 2.133149  [38400/60000]
loss: 2.105064  [44800/60000]
loss: 2.130375  [51200/60000]
loss: 2.018081  [57600/60000]
Test Error: 
 Accuracy: 41.3%, Avg loss: 0.032347 

Epoch 3
-------------------------------
loss: 2.092307  [    0/60000]
loss: 2.066211  [ 6400/60000]
loss: 2.029883  [12800/60000]
loss: 2.055547  [19200/60000]
loss: 1.928946  [25600/60000]
loss: 1.907717  [32000/600

In [12]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
